In [42]:
import csv
import random
import math

def loaddata(filename):
    lines = csv.reader(open(filename, "r"));
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]

    return dataset

def splitset(dataset, splitratio):
    trainsize = int(len(dataset) * splitratio);
    trainset = []
    copy = list(dataset)

    while len(trainset) < trainsize:
        index = random.randrange(len(copy));
        trainset.append(copy.pop(index))
        
    return [trainset, copy]


def standdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)
    
def calculateprobability(x, mean, standev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(standev,2))))
    return (1 / (math.sqrt(2*math.pi) * standev)) * exponent

def separatebyclass(dataset):
    separated = {}

    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
        
    return separated

def summarize(dataset): 
    summaries = [(mean(attribute), standdev(attribute)) for attribute in zip(*dataset)];
    del summaries[-1] 
    return summaries

def summarizebyclass(dataset):
    separated = separatebyclass(dataset);

    summaries = {}
    for classvalue, instances in separated.items():
        summaries[classvalue] = summarize(instances)

    return summaries

def mean(numbers):
  return sum(numbers)/float(len(numbers))



def calculateclassprobabilities(summaries, inputvector):
    probabilities = {}
    for classvalue, classsummaries in summaries.items():
        probabilities[classvalue] = 1
        for i in range(len(classsummaries)):
            mean, standev = classsummaries[i] 
            x = inputvector[i] 
            probabilities[classvalue] *= calculateprobability(x, mean, standev)
        return probabilities

def predict(summaries, inputvector): 
    probabilities = calculateclassprobabilities(summaries, inputvector)
    bestLabel, bestProb = None, -1

    for classvalue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classvalue

    return bestLabel

def getpredictions(summaries, testset):
    predictions = []

    for i in range(len(testset)):
        result = predict(summaries, testset[i])
        predictions.append(result)
        
    return predictions

def getaccuracy(testset, predictions):
    correct = 0
    for i in range(len(testset)):
        if testset[i][-1] == predictions[i]:
            correct += 1

    return (correct/float(len(testset))) * 100.0

splitratio = 0.72
dataset = loaddata("datasetBaysian.csv");

trainingset, testset = splitset(dataset, splitratio)
summaries = summarizebyclass(trainingset)
predictions = getpredictions(summaries, testset)
accuracy = getaccuracy(testset, predictions)
print(f'Accuracy with split ratio {splitratio} is : {accuracy}%')

Accuracy with split ratio 0.72 is : 63.888888888888886%
